<a href="https://colab.research.google.com/github/tayfununal/nku_lesson/blob/main/custom_loss_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def f(y, mu, sigma=20):
  return (1/(sigma * tf.math.sqrt(2 * np.pi)) * tf.math.exp( - (y - mu)**2 / (2 * sigma**2)))
  
def my_loss(y, y_pred):
  loss = -tf.reduce_sum(tf.math.log(f(y, y_pred)))
  return loss

In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
alpha = 32
model = Sequential()
model.add(Conv2D(alpha, (3, 3), input_shape=(32, 32, 3), strides=1, padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(2*alpha, (3, 3), strides=2))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(2*alpha, (3, 3), strides=1, padding="same"))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(4*alpha, (3, 3), strides=2))
model.add(Activation('relu'))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(64*alpha))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss=my_loss, optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, batch_size=512,
                    epochs=5, validation_split=0.2,
                    verbose=1)

In [ ]:
score = model.evaluate(X_test, Y_test,
                       batch_size=512, verbose=1)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()